# 🛣️ Road Damage Detection — YOLOv5 Training Notebook (Google Colab)

**Thesis:** Road Damage Detection Mobile App for LGU Road Surveys Using YOLOv5

This notebook trains a YOLOv5s model on the RDD2022 dataset **on Google Colab** with free GPU acceleration (NVIDIA T4/V100).

## ✨ Advantages of Google Colab
- **Free GPU:** NVIDIA T4 or V100 (saves money on cloud computing)
- **No setup required:** Pre-installed Python, CUDA, PyTorch
- **Easy collaboration:** Share notebook link with teammates
- **Auto-saves:** Progress saved to Google Drive
- **Integrated:** Direct access to Google Drive for data

## 🎯 Detects 4 Road Damage Types
- **D00** — Longitudinal Crack
- **D10** — Transverse Crack
- **D20** — Alligator Crack
- **D40** — Pothole

---

## Step 1: Connect to Google Drive & Check GPU

First, we'll mount your Google Drive and verify GPU access.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

In [ ]:
# Check GPU
import torch
import platform

print(f"Python: {platform.python_version()}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = "cuda"
else:
    print("⚠️ No GPU found! Training will be slower.")
    device = "cpu"

print(f"\n✅ Device selected: {device.upper()}")

## Step 2: Clone Repository & Install Dependencies

In [ ]:
import os
import subprocess
import sys

# Clone your repository (replace with your repo URL)
repo_url = "https://github.com/YOUR_USERNAME/road-damage-detection.git"
project_dir = "/content/road-damage-detection"

if not os.path.exists(project_dir):
    print(f"📥 Cloning repository...")
    subprocess.run(["git", "clone", repo_url, project_dir], check=True)
    print("✅ Repository cloned!")
else:
    print(f"✅ Repository already exists at {project_dir}")

os.chdir(project_dir)
print(f"\n📁 Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies
print("📦 Installing dependencies...")

# Core packages
subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                 "ultralytics", "opencv-python-headless", 
                 "Pillow", "pyyaml", "torch-class-imbalance"], check=False)

print("✅ Dependencies installed!")

## Step 3: Prepare Dataset

### Option A: Download from Kaggle (Recommended)
1. Get Kaggle API key: https://www.kaggle.com/settings/account
2. Upload `kaggle.json` to Colab
3. Run the cell below

In [ ]:
# Option A: Download from Kaggle
try:
    from google.colab import files
    print("📤 Upload your kaggle.json (from https://www.kaggle.com/settings/account)")
    uploaded = files.upload()
    
    # Setup Kaggle
    subprocess.run(["mkdir", "-p", "/root/.kaggle"], check=True)
    subprocess.run(["cp", "kaggle.json", "/root/.kaggle/"], check=True)
    subprocess.run(["chmod", "600", "/root/.kaggle/kaggle.json"], check=True)
    
    # Download dataset
    print("\n📥 Downloading RDD2022 dataset from Kaggle...")
    subprocess.run(["pip", "install", "-q", "kaggle"], check=True)
    subprocess.run([
        "kaggle", "datasets", "download", 
        "-d", "chitholian/road-damage-detection-rdd2022",
        "-p", "backend/data", "--unzip"
    ], check=True)
    
    print("✅ Dataset downloaded!")
except Exception as e:
    print(f"⚠️ Kaggle download skipped: {e}")
    print("\nUsing Option B instead...")

### Option B: Use Google Drive
1. Upload dataset to `My Drive/road-damage-detection/`
2. Run the cell below

In [ ]:
# Option B: Copy from Google Drive
drive_dataset_path = "/content/drive/MyDrive/road-damage-detection/data"
local_data_path = "backend/data"

if os.path.exists(drive_dataset_path):
    print(f"📋 Found dataset in Google Drive at: {drive_dataset_path}")
    print(f"📥 Copying to: {local_data_path}")
    
    os.makedirs(local_data_path, exist_ok=True)
    
    # Copy images
    if os.path.exists(os.path.join(drive_dataset_path, "images")):
        subprocess.run([
            "cp", "-r", 
            os.path.join(drive_dataset_path, "images"),
            local_data_path
        ], check=True)
        print("✅ Images copied")
    
    # Copy labels
    if os.path.exists(os.path.join(drive_dataset_path, "labels")):
        subprocess.run([
            "cp", "-r",
            os.path.join(drive_dataset_path, "labels"),
            local_data_path
        ], check=True)
        print("✅ Labels copied")
else:
    print(f"⚠️ Dataset not found at {drive_dataset_path}")
    print("\nInstructions:")
    print("1. Create folder: My Drive → New Folder → 'road-damage-detection'")
    print("2. Upload dataset inside this folder")
    print("3. Re-run this cell")

In [ ]:
# Setup data structure
print("📁 Setting up data structure...")

import sys
sys.path.insert(0, 'backend')

os.chdir("backend")
from download_dataset import setup_directory_structure, create_data_yaml

setup_directory_structure()
create_data_yaml()

print("✅ Data structure ready!")

# Check dataset
data_dir = "data"
train_images = os.path.join(data_dir, "images", "train")
train_labels = os.path.join(data_dir, "labels", "train")

if os.path.exists(train_images):
    num_images = len(os.listdir(train_images))
    print(f"\n📊 Training images: {num_images}")
if os.path.exists(train_labels):
    num_labels = len(os.listdir(train_labels))
    print(f"📊 Training labels: {num_labels}")

## Step 4: Train YOLOv5 Model

The model will train on the RDD2022 dataset. This takes ~2-4 hours with GPU.

In [ ]:
# Import YOLOv5
from ultralytics import YOLO
import torch

print("🚀 Starting YOLOv5 Training...")
print("=" * 70)

# Create model
model = YOLO('yolov5s.pt')  # Use yolov5s (smaller, faster)

# Training configuration
results = model.train(
    data='data/data.yaml',
    epochs=50,                    # Adjust based on GPU memory
    imgsz=640,
    batch=16,                    # Reduce if OOM: use 8 or 4
    patience=10,                 # Early stopping
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=4,
    cache='disk',               # Cache to disk (saves RAM)
    save=True,                  # Save checkpoints
    save_period=5,              # Save every 5 epochs
    verbose=True,
    amp=True,                   # Automatic mixed precision
    augment=True,               # Data augmentation
    hsv_h=0.015,                # HSV-color augmentation
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    dropout=0.0
)

print("\n" + "=" * 70)
print("✅ Training completed!")

In [ ]:
# Validate the model
print("📊 Validating model...")
val_results = model.val()
print("✅ Validation completed!")

## Step 5: Download Trained Model

In [ ]:
# Save best model to Google Drive for backup
import shutil

trained_model = "runs/detect/train/weights/best.pt"
models_dir = "models"

os.makedirs(models_dir, exist_ok=True)

if os.path.exists(trained_model):
    shutil.copy2(trained_model, os.path.join(models_dir, "best.pt"))
    print(f"✅ Model saved to: {os.path.join(models_dir, 'best.pt')}")
    
    # Also save to Google Drive
    drive_save_path = "/content/drive/MyDrive/road-damage-detection/best.pt"
    os.makedirs(os.path.dirname(drive_save_path), exist_ok=True)
    shutil.copy2(trained_model, drive_save_path)
    print(f"✅ Also saved to Google Drive: {drive_save_path}")
else:
    print(f"⚠️ Trained model not found at {trained_model}")

In [ ]:
# Download model files
from google.colab import files

print("📥 Downloading model files...")
files.download(os.path.join(models_dir, "best.pt"))
print("✅ Download started!")

## Step 6: Test the Model

Run inference on a sample image to verify the model works.

In [ ]:
# Test on a sample image
import cv2
from pathlib import Path

# Get first test image
test_images_dir = "data/images/val"
if os.path.exists(test_images_dir):
    test_images = list(Path(test_images_dir).glob("*.jpg")) + list(Path(test_images_dir).glob("*.png"))
    
    if test_images:
        test_image = str(test_images[0])
        print(f"🧪 Testing on: {test_image}")
        
        # Run inference
        results = model.predict(test_image, conf=0.25, imgsz=640)
        
        # Display results
        import matplotlib.pyplot as plt
        from PIL import Image
        
        result_image = results[0].plot()
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(result_image)
        plt.axis('off')
        plt.title('Road Damage Detection Results')
        plt.tight_layout()
        plt.show()
        
        print("\n✅ Test inference completed!")
    else:
        print("⚠️ No test images found in val/ directory")
else:
    print(f"⚠️ Test images directory not found: {test_images_dir}")

## Step 7: Training Summary

Display training metrics and results.

In [ ]:
# Display training results
train_dir = "runs/detect/train"
if os.path.exists(train_dir):
    print("📊 Training Results:")
    print("=" * 70)
    
    # List generated files
    for root, dirs, files in os.walk(train_dir):
        level = root.replace(train_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}📁 {os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Limit to 5 files per directory
            print(f'{subindent}📄 {file}')
        if len(files) > 5:
            print(f'{subindent}... and {len(files) - 5} more files')
else:
    print("⚠️ Training directory not found")

## Next Steps

1. ✅ Download `best.pt` from this notebook
2. 📱 Copy to mobile app: `mobile/models/best.pt`
3. 🚀 Deploy to your Flask backend
4. 📊 Monitor detection accuracy in production

---
**Created with ❤️ for road damage detection in LGU road surveys**